# 客户端

展示客户端与远程代理交互的示例。

您可以直接通过API与之交互

In [14]:
# !pip install pydantic==1.10.13

In [17]:
import requests

inputs = {"input": {"input": "Loi对猫有什么看法？"}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': {'output': '<|im_start|>assistant\nLoi是中国古代的一位哲学家，他对猫有着独特的看法。根据历史记载，Loi认为猫是一种非常聪明和独立的动物，他钦佩猫的自主性和自我保护的能力。\n\nLoi还注意到猫的狩猎技巧，他认为猫的狩猎能力是一种自然的、本能的能力，这种能力使得猫能够在自然界中生存和繁衍。\n\n此外，Loi还将猫的行为与人性的某些方面进行了比较。他认为，猫的独立性和自我保护的能力与人性的自我保护和独立性之间有着某种相似之处。\n\n总的来说，Loi对猫有着深刻的理解和钦佩，他将猫视为一种聪明、独立和有能力的动物。'},
 'metadata': {'run_id': '74b87cee-6441-46e4-b19a-188e9d404be2',
  'feedback_tokens': []}}

您还可以通过RemoteRunnable接口与之交互（用于其他链路中）

In [18]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

远程可运行对象与本地可运行对象具有相同的接口

In [19]:
await remote_runnable.ainvoke({"input": "你好!"})

{'output': '<|im_start|>assistant\n嗨！你好！我是助手，有什么需要我帮助的吗？<|im_end|>'}

In [20]:
remote_runnable.invoke({"input": "Loi对猫有什么看法？"})

{'output': '<|im_start|>assistant<|end_header_id|>\n\n根据《星球大战》中Loi的性格和行为，Loi对猫的看法似乎是中性的。他没有表现出对猫的特别喜爱或厌恶。在他与猫的互动中，他表现得友好和尊重，不会有任何负面或有害的行为。总的来说，Loi对猫保持着中立和尊重的态度。'}

## 流

请注意，流式传输在动作和观察之间交替进行。它不会单独流式传输单个令牌！如果您需要流式传输单个令牌，您将需要使用stream_log！

In [21]:
async for chunk in remote_runnable.astream({"input": "Loi对猫有何看法？然后，给我讲一个关于这个想法的故事。"}):
    print('--')
    print(chunk)

--
{'output': '<|im_start|>assistant\nLoi认为猫是独立的灵魂，拥有自己的想法和感受。它们是自私的，冷漠的，甚至有点固执，但同时也非常敏感和观察力强。\n\n这里有一个故事关于Loi的猫的想法：\n\nLoi有一只叫做Momo的猫。Momo是一只非常独立的猫，它们喜欢独自行动，很少需要人的陪伴。但是有一天，Loi注意到Momo似乎有些不舒服，总是躲在角落里不肯出来。Loi试图靠近Momo，但Momo总是把头扭过去，似乎不想被打扰。\n\nLoi觉得Momo可能是因为一些过去的经历而变得这样孤僻。于是，他决定试着和Momo沟通。他坐下来，轻轻地摸着Momo的头，温柔地对它说话。慢慢地，Momo开始放松下来，甚至开始主动靠近Loi。\n\nLoi意识到，Momo并不是因为自私或冷漠，而是因为它曾经受过伤害，需要时间来恢复。Loi决定给Momo更多的空间和时间，让它自己来决定是否想要靠近他。\n\n从那天起，Loi和Momo的关系发生了变化。Momo开始变得更加信任Loi，甚至会主动爬到Loi的怀里。Loi也学会了尊重Momo的独立性，不再强迫它做自己不想做的事情。\n\n这个故事讲述了Loi对猫的看法：即使猫看起来独立和冷漠，但它们也需要关心和理解。通过尊重和沟通，人类可以与猫建立更深层次的关系。', 'messages': [AIMessage(content='<|im_start|>assistant\nLoi认为猫是独立的灵魂，拥有自己的想法和感受。它们是自私的，冷漠的，甚至有点固执，但同时也非常敏感和观察力强。\n\n这里有一个故事关于Loi的猫的想法：\n\nLoi有一只叫做Momo的猫。Momo是一只非常独立的猫，它们喜欢独自行动，很少需要人的陪伴。但是有一天，Loi注意到Momo似乎有些不舒服，总是躲在角落里不肯出来。Loi试图靠近Momo，但Momo总是把头扭过去，似乎不想被打扰。\n\nLoi觉得Momo可能是因为一些过去的经历而变得这样孤僻。于是，他决定试着和Momo沟通。他坐下来，轻轻地摸着Momo的头，温柔地对它说话。慢慢地，Momo开始放松下来，甚至开始主动靠近Loi。\n\nLoi意识到，Momo并不是因为自私或冷漠，而是因为它曾经受过伤害，需要时间来恢复。Loi决定给Momo更多的空间和时间，让它自己

## 流事件

客户端正在寻找名为 `agent` 的可运行对象，用于链路事件。这个名称是在服务器端使用 `runnable.with_config({"run_name": "agent"}` 定义的。

In [22]:
async for event in remote_runnable.astream_events(
    {"input": "Loi对猫有何看法？然后，给我讲一个关于这个想法的故事。"},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: agent with input: {'input': 'Loi对猫有何看法？然后，给我讲一个关于这个想法的故事。'}
这个|问题|涉|及|对|Lo|i|的|个人|观|点|的|想|象|，因为|我|无法|从|上|下|文|中|获得|有关|Lo|i|对|猫|的|想|法|的|信息|。|以下|是|基于|对|Lo|i|想|法|的|想|象|的|回答|。

|Lo|i|认为|猫|是|世界|上|最|奇|怪|的|生物|之一|。|它们|优|雅|、|神|秘|，有|时|甚至|有些|吓|人|。|对于|Lo|i|来说|，|猫|的|神|秘|感|源|于|它们|的|行为|，|态|度|和|举|止|。|Lo|i|认为|猫|的|这种|神|秘|感|是一|种|独|特|的|魅|力|，它|让|人|无法|抗|拒|。|例如|，|Lo|i|一|天|晚|上|醒|来|，|发现|她的|猫|，|L|una|，|正|坐在|她的|床|上|，|盯|着|她|看|。她|突然|感到|有些|紧|张|，但|同时|又|被|L|una|的|神|秘|气|质|所|吸|引|。|Lo|i|认为|猫|的|这种|神|秘|感|是|它们|独|特|的|特|征|之一|，这|让|它们|如此|迷|人|和|让|人|着|迷|。|
--
Done agent: agent with output: 这个问题涉及对Loi的个人观点的想象，因为我无法从上下文中获得有关Loi对猫的想法的信息。以下是基于对Loi想法的想象的回答。

Loi认为猫是世界上最奇怪的生物之一。它们优雅、神秘，有时甚至有些吓人。对于Loi来说，猫的神秘感源于它们的行为，态度和举止。Loi认为猫的这种神秘感是一种独特的魅力，它让人无法抗拒。例如，Loi一天晚上醒来，发现她的猫，Luna，正坐在她的床上，盯着她看。她突然感到有些紧张，但同时又被Luna的神秘气质所吸引。Loi认为猫的这种神秘感是它们独特的特征之一，这让它们如此迷人和让人着迷。


## 流日志

如果您需要访问代理的单个LLM令牌，请使用 `astream_log`。请确保您在LLM上设置了 **streaming=True**（参见服务器代码）。要使此功能正常工作，LLM还必须支持流式传输！

In [23]:
async for chunk in remote_runnable.astream_log({"input": "Loi对猫有何看法？"}):
    print('--')
    print(chunk)

--
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'c219ede8-46eb-4b62-99e5-8a4a923aa78d',
            'logs': {},
            'name': 'agent',
            'streamed_output': [],
            'type': 'chain'}})
--
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableSequence',
  'value': {'end_time': None,
            'final_output': None,
            'id': '074b7eb9-acb8-472e-bd9b-e8f0f08019dd',
            'metadata': {'__langserve_endpoint': 'stream_log',
                         '__langserve_version': '0.2.3',
                         '__useragent': 'python-httpx/0.27.0'},
            'name': 'RunnableSequence',
            'start_time': '2024-09-05T06:17:14.459+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': [],
            'type': 'chain'}})
--
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel<input,agent_scratchpad>',
  'value': {'end_time': None,
            'final_outpu